In [1]:
!pip uninstall -y transformers
!pip install transformers==4.44.2 datasets==2.19.0

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)


In [2]:
import transformers

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
from datasets import load_dataset

In [4]:
from transformers import DataCollatorForWholeWordMask, AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments, pipeline, BertTokenizerFast, BertTokenizer

In [5]:
from tokenizers import BertWordPieceTokenizer

In [6]:
import requests

In [7]:
import os
import shutil

In [8]:
# MODEL_NAME = "DeepPavlov/rubert-base-cased" # или "DeepPavlov/bert-base-bg-cs-pl-ru-cased"
MODEL_NAME = "DeepPavlov/bert-base-bg-cs-pl-ru-cased"

In [9]:
OUTPUT_DIR = "old_russian_slavic_bert_finetune"

In [10]:
TRAIN_FILE = "final_dataset_ready.txt"

In [11]:
VOCAB_DIR = "./custom_vocab_dir"

In [12]:
ORIGINAL_FILE = "vocab_extended.txt"

In [13]:
VOCAB_SIZE_LIMIT = 5000

In [14]:
vocab_url = "https://huggingface.co/DeepPavlov/bert-base-bg-cs-pl-ru-cased/resolve/main/vocab.txt"
try:
    r = requests.get(vocab_url)
    original_vocab = r.text.splitlines()
    print(f" Downloader.{len(original_vocab)} tokens in the dataset")
except Exception as e:
    raise ValueError(f"Error: {e}")

 Downloader.119547 tokens in the dataset


In [15]:
with open(TRAIN_FILE, 'r', encoding='utf-8') as f:
    text_content = f.read()

In [16]:
existing_vocab_set = set(original_vocab)
tokens_to_append = []

In [17]:
unique_chars = set(text_content)
chars_found = 0

In [18]:
for c in unique_chars:
    if c.strip(): # Пропускаем пробелы
        # 1. Добавляем саму букву (для начала слова)
        if c not in existing_vocab_set and c not in tokens_to_append:
            tokens_to_append.append(c)
            chars_found += 1

        # 2. ВАЖНО: Добавляем вариант с ## (для середины слова)
        sub_c = "##" + c
        if sub_c not in existing_vocab_set and sub_c not in tokens_to_append:
            tokens_to_append.append(sub_c)

In [19]:
print(f"Added letters/symbols: {chars_found} (Examples: {tokens_to_append})")

Added letters/symbols: 129 (Examples: ['ӱ', '##ӱ', 'ꙃ', '##ꙃ', '\uf4a5', '##\uf4a5', '‐', '##‐', '\uf4a4', '##\uf4a4', '҆', '##҆', '҅', '##҅', '͆', '##͆', '̋', '##̋', '¦', '##¦', '꙯', '##꙯', 'ѽ', '##ѽ', '\uf467', '##\uf467', '̾', '##̾', '\uf074', '##\uf074', 'Ѽ', '##Ѽ', '`', '##`', '͘', '##͘', 'ѡ', '##ѡ', '꙳', '##꙳', 'ȣ', '##ȣ', '\uf245', '##\uf245', 'ⷩ', '##ⷩ', '\uf13f', '##\uf13f', '\uf2d1', '##\uf2d1', '҂', '##҂', '⋮', '##⋮', 'ѕ', '##ѕ', 'Ѳ', '##Ѳ', 'ꙁ', '##ꙁ', '\uf1c0', '##\uf1c0', 'ꙋ', '##ꙋ', 'ѵ', '##ѵ', '͞', '##͞', 'ӏ', '##ӏ', 'ⱕ', '##ⱕ', 'ѩ', '##ѩ', '̕', '##̕', 'Ѻ', '##Ѻ', '∙', '##∙', 'ⰹ', '##ⰹ', 'ⱔ', '##ⱔ', '̒', '##̒', '“', '##“', '͗', '##͗', 'Ѥ', '##Ѥ', 'ѯ', '##ѯ', '̏', '##̏', 'ӑ', '##ӑ', 'ѧ', '##ѧ', 'ⷮ', '##ⷮ', 'ᲂ', '##ᲂ', '̈', '##̈', 'ѿ', '##ѿ', '⁘', '##⁘', 'Ї', '##Ї', '\uf130', '##\uf130', '҃', '##҃', '\uf2b4', '##\uf2b4', '̓', '##̓', '\uf47e', '##\uf47e', 'ѫ', '##ѫ', 'Ѱ', '##Ѱ', 'ꙙ', '##ꙙ', '\uf222', '##\uf222', 'ꙇ', '##ꙇ', 'Ѡ', '##Ѡ', 'ⷪ', '##ⷪ', 'ѭ', '##ѭ', 'Ѕ', '##Ѕ', '

## Adding vocabulary / words

In [20]:
from collections import Counter

In [21]:
words = text_content.split()
word_counts = Counter(words)

In [22]:
forbidden = set(".,;!?:()[]\"'«»-\n\r\t")
words_found = 0

In [23]:
for w, c in word_counts.most_common(VOCAB_SIZE_LIMIT + 2000): # Берем с запасом
    # Условия:
    # 1. Слова нет в словаре
    # 2. Длина > 1 (буквы уже добавили)
    # 3. Нет запрещенных знаков внутри
    if w not in existing_vocab_set and len(w) > 1:
        if not any(bad in w for bad in forbidden):
            tokens_to_append.append(w)
            words_found += 1
            if words_found >= VOCAB_SIZE_LIMIT:
                break

In [24]:
print(f"Adding words: {words_found}")
print(f"All tokens: {len(tokens_to_append)}")

Adding words: 5000
All tokens: 5258


In [25]:
print(f"Creating a file {ORIGINAL_FILE}...")
full_vocab = original_vocab + tokens_to_append

with open(ORIGINAL_FILE, "w", encoding="utf-8") as f:
    for token in full_vocab:
        f.write(token + "\n")

Creating a file vocab_extended.txt...


In [26]:
os.makedirs(VOCAB_DIR, exist_ok=True)
dest_file = os.path.join(VOCAB_DIR, "vocab.txt")
shutil.copy(ORIGINAL_FILE, dest_file)

'./custom_vocab_dir/vocab.txt'

In [27]:
tokenizer = BertTokenizer.from_pretrained(
      VOCAB_DIR,
      do_lower_case=False,
      unk_token="[UNK]"
  )

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [28]:
len(tokenizer)

124805

In [29]:
print("\n>>> 🧪 ТЕСТ ТОКЕНИЗАЦИИ:")
test_phrase = "поклоно ѿ онѳима ко осподину"
tokens = tokenizer.tokenize(test_phrase)
print(f"   Фраза: '{test_phrase}'")
print(f"   Токены: {tokens}")


>>> 🧪 ТЕСТ ТОКЕНИЗАЦИИ:
   Фраза: 'поклоно ѿ онѳима ко осподину'
   Токены: ['поклоно', 'ѿ', 'он', '##ѳ', '##им', '##а', 'ко', 'оспо', '##дин', '##у']


In [30]:
save_path = "./old_rus_tokenizer"
tokenizer.save_pretrained(save_path)

('./old_rus_tokenizer/tokenizer_config.json',
 './old_rus_tokenizer/special_tokens_map.json',
 './old_rus_tokenizer/vocab.txt',
 './old_rus_tokenizer/added_tokens.json')

## Loading everything

In [31]:
tokenizer = AutoTokenizer.from_pretrained("./old_rus_tokenizer", use_fast=True)

In [32]:
model = AutoModelForMaskedLM.from_pretrained("DeepPavlov/bert-base-bg-cs-pl-ru-cased")

In [33]:
model.resize_token_embeddings(len(tokenizer))

Embedding(124805, 768, padding_idx=0)

In [34]:
dataset = load_dataset("text", data_files={"train": TRAIN_FILE})

Generating train split: 0 examples [00:00, ? examples/s]

In [35]:
def tokenize_function(examples):
    # Токенизируем
    result = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        return_special_tokens_mask=True
    )

    # ГЛАВНАЯ МАГИЯ: Добавляем word_ids для каждого примера
    # Это позволяет коллатору понять, какие токены принадлежат одному слову
    all_word_ids = []
    for i in range(len(examples["text"])):
        word_ids = result.word_ids(batch_index=i)
        # Заменяем None (спецтокены) на -1 для совместимости
        processed_word_ids = [w if w is not None else -100 for w in word_ids]
        all_word_ids.append(processed_word_ids)

    result["word_ids"] = all_word_ids
    return result

In [36]:
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=4,        # Используем 4 ядра процессора для ускорения
    remove_columns=["text"] # Удаляем исходный текст, оставляем только цифры
)

Map (num_proc=4):   0%|          | 0/71255 [00:00<?, ? examples/s]

In [37]:
block_size = 512
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

In [38]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/71255 [00:00<?, ? examples/s]

In [39]:
print(f"Total blocks for training: {len(lm_datasets['train'])}")

Total blocks for training: 3961


In [40]:
print(f"   🧐 Проверка колонок: {lm_datasets['train'].column_names}")
if "word_ids" not in lm_datasets['train'].column_names:
    raise ValueError("❌ Ошибка! Колонка word_ids исчезла при процессинге.")

   🧐 Проверка колонок: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask', 'word_ids']


In [41]:
data_collator = DataCollatorForWholeWordMask(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [45]:
training_args = TrainingArguments(
    save_safetensors=False,
    remove_unused_columns=False,
    output_dir=OUTPUT_DIR,
    num_train_epochs=15,             # Ставим 15 эпох для надежности
    per_device_train_batch_size=8,   # 8 для T4 GPU (если вылетает OOM, ставь 4)
    gradient_accumulation_steps=2,   # Накапливаем градиент (эффективный батч = 16)
    learning_rate=5e-5,
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=2,
    fp16=True,                       # Включаем Mixed Precision (ускорение)
    logging_steps=50,
    report_to="none")

In [46]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    tokenizer =tokenizer # Передаем токенизатор, чтобы он сохранился в финале
)

In [47]:
trainer.train()

Step,Training Loss
50,4.453300
100,4.312600
150,4.278100
200,4.173500
250,4.069200
300,3.998500
350,3.984600


KeyboardInterrupt: 

In [ ]:
trainer.save_model(OUTPUT_DIR)

In [ ]:
print("\n>>> ТЕСТИРУЕМ МОДЕЛЬ:")
fill_mask = pipeline("fill-mask", model=OUTPUT_DIR, tokenizer=OUTPUT_DIR, device=0)

test_phrase = "поклоно ѿ [MASK] ко осподину"
print(f"Запрос: {test_phrase}")
res = fill_mask(test_phrase)
for r in res:
    print(f"🔹 {r['token_str']} ({r['score']:.1%})")